In [21]:
import json
import numpy as np
import os
from soma import aims

from sulci.registration.spam import spam_register

import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

from soma.aimsalgo import MorphoGreyLevel_S16

# Global static variables
_AIMS_BINARY_ONE = 32767
_dilation = 0
_threshold = 3

# launching anatomist
a = ana.Anatomist()

In [22]:
def dilate(mask, radius=_dilation):
    """Makes a dilation radius _dilation, in mm
    """
    arr = mask.np
    # Binarization of mask
    arr[arr < 1] = 0
    if radius > 0:
        arr[arr >= 1] = _AIMS_BINARY_ONE
        # Dilates initial volume of 10 mm
        morpho = MorphoGreyLevel_S16()
        dilate = morpho.doDilation(mask, radius)
        arr_dilate = dilate.np
        arr_dilate[arr_dilate >= 1] = 1
        return dilate
    else:
        arr[arr >= 1] = 1
        return mask

In [23]:
spam_file = '/neurospin/dico/data/deep_folding/current/mask/2mm/regions/L/Sc.Cal.-S.Li._left.nii.gz'
skel_f = '/neurospin/dico/data/deep_folding/current/datasets/pclean/binarized_skeletons/L/Lbinarized_skeleton_ammon.nii.gz' 

# Reads spam and skeleton files
mask_result = aims.read(spam_file)
skel_data = aims.read(skel_f)
skel_data.np[:] = (skel_data.np > 0).astype(np.int16)

# Makes binarization and dilation on spam
mask_result[mask_result.np <= _threshold] = 0
mask_result.np[:] = dilate(mask_result).np

In [24]:
mask_result.shape

(96, 114, 96, 1)

In [25]:
# Masks skeleton data with dilated spam
# Translates
skel_data.np[mask_result.np <= 0] = 0
# skel_data_not_translated = aims.Volume(skel_data)
skel_data.np[:] = np.roll(skel_data.np, 5, axis=0)
aims.write(skel_data, "/tmp/skel_before.nii.gz")

In [26]:
# Reads initial spam volume and transforms it to proba
spam_vol = aims.read(spam_file, dtype="Volume_FLOAT")
spam_vol.np[:] = spam_vol.np/61

In [27]:
np.unique(spam_vol.np)

array([0.        , 0.01639344, 0.03278688, 0.04918033, 0.06557377,
       0.08196721, 0.09836066, 0.1147541 , 0.13114753, 0.14754099,
       0.16393442, 0.18032786, 0.19672132, 0.21311475, 0.22950819,
       0.24590164, 0.26229507, 0.27868852, 0.29508197, 0.3114754 ,
       0.32786885, 0.3442623 , 0.36065573, 0.37704918, 0.39344263,
       0.40983605, 0.4262295 , 0.44262296, 0.45901638, 0.47540984,
       0.4918033 , 0.5081967 , 0.52459013, 0.5409836 , 0.55737704],
      dtype=float32)

In [28]:
np.unique(skel_data, return_counts=True)

(array([0, 1], dtype=int16), array([1050325,     299]))

In [29]:
# Makes realignment
out_tr = spam_register(spam_vol,
                       skel_data,
                       do_mask=False,
                       R_angle_var=np.pi / 8,
                       t_var=5.,
                       verbose=False,
                       in_log=False,
                       calibrate_distrib=30)
# out_tr.setTranslation((5, 0, 0))
aims.write(out_tr, '/tmp/transform.trm')

In [30]:
# Prints transformation matrix
out_tr.np

array([[ 9.47544873e-01,  2.98102915e-01, -1.15296863e-01,
        -4.04252396e+01],
       [-2.98106134e-01,  9.54370081e-01,  1.76202394e-02,
         3.87702484e+01],
       [ 1.15288526e-01,  1.76747367e-02,  9.93174791e-01,
        -1.52747498e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]], dtype=float32)

In [31]:
# Applies the realignment
os.system(f"AimsApplyTransform -i /tmp/skel_before.nii.gz -o /tmp/skel_realigned.nii.gz -m /tmp/transform.trm")

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


loading direct transformations
Output dimensions: 96, 114, 96
Output voxel size: 2, 2, 2 mm
Resampling carto_volume of S16...   0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 %


0

In [32]:
before = aims.read("/tmp/skel_before.nii.gz")
after = aims.read("/tmp/skel_realigned.nii.gz")
print(np.unique(before.np, return_counts=True))
print(np.unique(after.np, return_counts=True))
print(before.shape)
print(after.shape)

(array([0, 1], dtype=int16), array([1050325,     299]))
(array([0, 1], dtype=int16), array([1050396,     228]))
(96, 114, 96, 1)
(96, 114, 96, 1)


In [33]:
after.header()['voxel_size'].np

array([2., 2., 2., 1.], dtype=float32)

In [34]:
np.array_equal(before, after)

False

In [35]:
# Visualization
spam = a.loadObject(spam_file)
skel = a.loadObject("/tmp/skel_before.nii.gz")
realigned = a.loadObject("/tmp/skel_realigned.nii.gz")
skel.setPalette("BLUE-lfusion")
realigned.setPalette("RED-lfusion")
w = a.createWindow('Sagittal')
w.addObjects(spam)
w.addObjects(skel)
w.addObjects(realigned)

observable 0x603ce2578fd0(N9anatomist7AVolumeIsEE) could not be removed from observer 0x603ce426efd8 (N9anatomist8Fusion2DE)
observable 0x603ce40f8da0(N9anatomist7AVolumeIsEE) could not be removed from observer 0x603ce3f00968 (N9anatomist8Fusion2DE)


In [36]:
dict(mask_result.header())

{'volume_dimension': <soma.aims.vector_S32 at 0x717944fe56c0>,
 'sizeX': 96,
 'sizeY': 114,
 'sizeZ': 96,
 'sizeT': 1,
 'referential': '84b1989b-eb68-8665-0049-8feaf3c22679',
 'disk_data_type': 'S16',
 'bits_allocated': 16,
 'data_type': 'S16',
 'scale_factor_applied': False,
 'possible_data_types': <soma.aims.vector_STRING at 0x717944fe5c60>,
 'cal_min': 0.0,
 'cal_max': 0.0,
 'freq_dim': 0,
 'phase_dim': 0,
 'slice_dim': 0,
 'slice_code': 0,
 'slice_start': 0,
 'slice_end': 0,
 'slice_duration': 0.0,
 'storage_to_memory': <soma.aims.vector_FLOAT at 0x717944fe5ab0>,
 'voxel_size': <soma.aims.vector_FLOAT at 0x717944fe5900>,
 'tr': 1.0,
 'referentials': <soma.aims.vector_STRING at 0x717944fe5870>,
 'transformations': <soma.aims.vector_vector_FLOAT at 0x717944fe5750>,
 'toffset': 0.0,
 'xyz_units': 0,
 'time_units': 0,
 'descrip': '',
 'aux_file': '',
 'nifti_type': 1,
 'object_type': 'Volume',
 'file_type': 'NIFTI-1'}

In [37]:
print(before.header()['referentials'])
print(after.header()['referentials'])
print(mask_result.header()['referentials'])

["Talairach-MNI template-SPM"]
["Talairach-MNI template-SPM", "2053787e-9cee-11ef-976a-a4bb6db7a741"]
["Talairach-MNI template-SPM"]


Position : 108, 28.0605, 11.638, 0
Position : 106, 151.775, 164.745, 0
Position : 106, 146.627, 116.422, 0
